In [1]:
!pip install -q -U trl transformers accelerate git+https://github.com/huggingface/peft.git
!pip install -q datasets bitsandbytes einops wandb


  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.3/155.3 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 91.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 34.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.7/536.7 kB 39.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.8/79.8 kB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 17.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 19.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 89.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 195.

In [2]:
import os
import transformers
import torch
from google.colab import userdata
from datasets import load_dataset
from trl import SFTTrainer
from peft import LoraConfig
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import BitsAndBytesConfig, GemmaTokenizer

In [3]:
os.environ["Huggingface"] = userdata.get('Huggingface')

In [4]:
model_id = "google/gemma-2b"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    trust_remote_code=True,
    token=os.environ['Huggingface']
)

model.config.use_cache = False

config.json:   0%|          | 0.00/627 [00:00<?, ?B/s]

`low_cpu_mem_usage` was None, now set to True since model is quantized.


model.safetensors.index.json:   0%|          | 0.00/13.5k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/67.1M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

In [5]:
os.environ["WANDB_DISABLED"] = "false"

In [6]:
tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True,token=os.environ['Huggingface'])
tokenizer.pad_token = tokenizer.eos_token
max_length = 512  # Set your desired maximum sequence length
tokenizer.padding_side = 'right'
tokenizer.max_length = max_length

tokenizer_config.json:   0%|          | 0.00/1.11k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/555 [00:00<?, ?B/s]

In [7]:
from datasets import load_dataset
import numpy as np
# Load the CSV dataset
dataset = load_dataset("csv", data_files="/content/dataset_cleaned.csv", split='train')

# Convert the dataset to a Hugging Face Dataset
hf_dataset = np.array(dataset[0])

# Access the dataset through indices as a normal array
#example = hf_dataset[0]
print(dataset[21])

Generating train split: 0 examples [00:00, ? examples/s]

{'Title': 'For more, refer to the Advantages and Disadvantages of Mesh Topology.'}


In [8]:
from peft import LoraConfig, get_peft_model

lora_alpha = 16
lora_dropout = 0.1
lora_r = 64

peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM"
)

In [9]:
from transformers import TrainingArguments

output_dir = "./results"
per_device_train_batch_size = 1
gradient_accumulation_steps = 1
optim = "paged_adamw_32bit"
save_steps = 100
logging_steps = 10
learning_rate = 2e-4
max_grad_norm = 0.3
max_steps = 200
warmup_ratio = 0.03
lr_scheduler_type = "constant"

training_arguments = TrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    fp16=True,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=True,
    lr_scheduler_type=lr_scheduler_type,
    report_to="wandb"
)

In [ ]:
def formatting_func(example):
    text = f"Quote: {example['quote'][0]}\nAuthor: {example['author'][0]}"
    return [text]

In [10]:
from trl import SFTTrainer

max_seq_length = 512

trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_config,
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_arguments,
    dataset_batch_size=16,
    dataset_text_field = 'Title'
)

Map:   0%|          | 0/718 [00:00<?, ? examples/s]

In [ ]:
import pandas as pd
df = pd.read_csv('/content/dataset_cleaned.csv')
df.head()

,Title
0,Rule 1) Send one data packet at a time. Rule 2...
1,Rule 1) Send acknowledgement after receiving a...
2,3. Delayed Acknowledgement/Data: After a timeo...
3,The above 3 problems are resolved by Stop and ...
4,3. Delayed Acknowledgement: This is resolved b...


In [11]:
for name, module in trainer.model.named_modules():
    if "norm" in name:
        module = module.to(torch.float16)

In [12]:
trainer.train()

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Step,Training Loss
10,2.252600
20,2.601500
30,2.878400
40,2.951500
50,3.820800
60,2.526500
70,2.718800
80,2.758600
90,2.637700
100,3.028900


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/peft/utils/other.py:588: UserWarning: Unable to fetch remote file due to the following error 401 Client Error. (Request ID: Root=1-65db641c-50f97efa1b7e5a105ac68612;0ecac225-f6aa-4abc-8ed9-040c99f0eb7a)

Cannot access gated repo for url https://huggingface.co/google/gemma-2b/resolve/main/config.json.
Repo model google/gemma-2b is gated. You must be authenticated to access it. - silently ignoring the lookup for the file config.json in goo

TrainOutput(global_step=200, training_loss=2.652292642593384, metrics={'train_runtime': 90.0241, 'train_samples_per_second': 2.222, 'train_steps_per_second': 2.222, 'total_flos': 140541022310400.0, 'train_loss': 2.652292642593384, 'epoch': 0.28})

In [13]:
trainer.save_model("/content/pretrained_gemma_2b")

/usr/local/lib/python3.10/dist-packages/peft/utils/other.py:588: UserWarning: Unable to fetch remote file due to the following error 401 Client Error. (Request ID: Root=1-65db6464-56b8babf6f05c8d612247be5;2c07d45e-082c-4a7d-9f26-8efc51670df4)

Cannot access gated repo for url https://huggingface.co/google/gemma-2b/resolve/main/config.json.
Repo model google/gemma-2b is gated. You must be authenticated to access it. - silently ignoring the lookup for the file config.json in google/gemma-2b.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/peft/utils/save_and_load.py:154: UserWarning: Could not find a config file in google/gemma-2b - will assume that the vocabulary was not modified.
  warnings.warn(


In [15]:
loaded_model = '/content/pretrained_gemma_2b'
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)

model_loaded = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    trust_remote_code=True
)
model_loaded.config.use_cache = False

`low_cpu_mem_usage` was None, now set to True since model is quantized.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [16]:
tokenizer = AutoTokenizer.from_pretrained(model_id, token=os.environ['Huggingface'])

In [25]:
text = "Explain the setup, data transfer and tear-down phases"
device = "cuda:0"
inputs = tokenizer(text, return_tensors="pt").to(device)

outputs = model.generate(**inputs, max_new_tokens=100)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Explain the setup, data transfer and tear-down phases of the TCP connection.

Answer:

The TCP connection setup phase involves establishing a connection between two devices, such as a computer and a printer. This phase involves the following steps:

1. Establishing a connection: The devices exchange information to determine if they can establish a connection. This information includes the device's IP address, port number, and other details.

2. Establishing a handshake: The devices then exchange a series of messages to confirm the connection and establish a common understanding of the
